In [1]:
import pandas as pd
import pandas_datareader.data as web
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
url = 'https://www.sashares.co.za/shares-list/#gs.dct9y1'
response = requests.get(url)

In [3]:
website = response.text
soup = BeautifulSoup(website, features = "lxml")
table = soup.find('table')

In [5]:
rows = len(table.findAll('tr')) - 1
columns = 2 #len(table.findAll('tr')[0])
print((rows, columns))

(486, 2)


In [6]:
col_names = [table.findAll('tr')[0].find_all('th')[i].text for i in range(columns)]
col_names

['JSE Code', 'Share']

In [9]:
companies_dict = {col: [] for col in col_names}

for col in range(columns):
    for row in range(1, rows + 1):
        
        if col == 1:
            value = table.findAll('tr')[row].find_all('td')[col].text
            companies_dict[col_names[col]].append(value)
        elif col == 0:
            value = table.findAll('tr')[row].find_all('td')[col].text.split('.')[0]
            companies_dict[col_names[col]].append(value + '.JO')

In [10]:
Companies = pd.DataFrame.from_dict(companies_dict)
Companies

,JSE Code,Share
0,4SI.JO,4SIGHT
1,ABG.JO,ABSA
2,ABSP.JO,ABSABANK-P
3,ACE.JO,ACCENT
4,ACG.JO,ANCHOR
...,...,...
481,YYLBEE.JO,YEBOYETHU
482,ZCL.JO,ZARCLEAR
483,ZED.JO,ZEDER
484,ZPLP.JO,ZPLP


In [12]:
Prices = {}
symbols = Companies['JSE Code']
start = datetime(2015, 1, 1).date()
end = end = datetime.now().date()


symbols_not_found = []

for i, symbol in enumerate(symbols):
    try:

        data = web.DataReader(
            name = symbol, 
            data_source = 'yahoo',
            start = start,
            end = end)\
            .rename(columns = {'Open': 'open',
                               'High': 'high',
                               'Low': 'low',
                               'Close': 'close',
                               'Adj Close': 'adj_close',
                               'Volume': 'volume'})

        Prices[symbol] = data
    except Exception:
        symbols_not_found.append(symbol)
        print(f'{i}: {symbol} not available in database')

103: CVW.JO not available in database
118: ECSD.JO not available in database
119: ECSD7.JO not available in database
120: ECSG.JO not available in database
121: ECSP1.JO not available in database
122: ECSP21.JO not available in database
123: ECSP24.JO not available in database
136: ERN.JO not available in database
155: FDP.JO not available in database
161: FSE.JO not available in database
162: FSEO1.JO not available in database
196: IBLVR2.JO not available in database
199: IBRP2.JO not available in database
203: IMCB22.JO not available in database
219: JBL.JO not available in database
228: KP2.JO not available in database
236: LEW.JO not available in database
238: LNF.JO not available in database
240: LUX.JO not available in database
243: MCG.JO not available in database
246: MED.JO not available in database
255: MRF.JO not available in database
256: MRI.JO not available in database
261: MTA.JO not available in database
262: MTH.JO not available in database
264: MTN.JO not available in

In [13]:
len(Prices.keys())

431

In [14]:
Companies.to_pickle('Companies.pkl')

In [15]:
import pickle
pickle.dump(Prices, open('Prices.pkl', 'wb'))